### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
student_data.head()
#school_data.head()
#school_data_complete.tail()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
schoolCount = school_data['School ID'].count()
studentCount = student_data['Student ID'].count()
totalBudget = school_data['budget'].sum()
meanMathScore = round(student_data['math_score'].mean(),1)
meanReadScore = round(student_data['reading_score'].mean(),1)
overallPassRate = round((meanMathScore + meanReadScore)/2,0)
perCPassMath = round(len(student_data.loc[student_data['math_score'] >= 70, 'Student ID']) / studentCount,2)*100
perCPassRead = round(len(student_data.loc[student_data['reading_score'] >= 70, 'Student ID'])/studentCount,2)*100

districtSummaryDF = pd.DataFrame({'School_Count':[schoolCount],'Student_Count':[studentCount],
                                  'Total_Budget':[totalBudget], 'Avg_Math_Score':[meanMathScore],
                                  'Avg_Reading_Score':[meanReadScore], 'Overall_Pass_Rate':[overallPassRate],
                                  'Perc_Passing_Math':[perCPassMath], 'Perc_Passing_Reading':[perCPassRead]})

districtSummaryDF
# Clean Up Formatting in new dataframe
 #Relabel columns
districtSummaryDF_Fmt = districtSummaryDF.rename(columns={'School_Count':'Count of Schools','Student_Count':'Count of Students',
                                                                'Total_Budget':'Total Budget', 'Avg_Math_Score':'Avg Math Score',
                                                                'Avg_Reading_Score': 'Avg Reading Score', 'Overall_Pass_Rate':'Overall Pass Rate',
                                                                'Perc_Passing_Math':'% Passing Math','Perc_Passing_Reading': '% Passing Reading' })
 #Format values
districtSummaryDF_Fmt["Count of Students"] = districtSummaryDF_Fmt["Count of Students"].apply("{:,.0f}".format)
districtSummaryDF_Fmt["Total Budget"] = districtSummaryDF_Fmt["Total Budget"].astype(float).apply("${:,.0f}".format)
districtSummaryDF_Fmt["Overall Pass Rate"] = districtSummaryDF_Fmt["Overall Pass Rate"].astype(float).apply("{:,.0f}%".format)
districtSummaryDF_Fmt["% Passing Math"] = districtSummaryDF_Fmt["% Passing Math"].astype(float).apply("{:,.0f}%".format)
districtSummaryDF_Fmt["% Passing Reading"] = districtSummaryDF_Fmt["% Passing Reading"].apply("{:,.0f}%".format)

districtSummaryDF_Fmt

,Count of Schools,Count of Students,Total Budget,Avg Math Score,Avg Reading Score,Overall Pass Rate,% Passing Math,% Passing Reading
0,15,"39,170","$24,649,428",79.0,81.9,80%,75%,86%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
#add boolean columns to support %passing math and %passing reading
#the mean method will see the boolean as 0/1..

school_data_complete['passingMath'] = np.where(school_data_complete.math_score >= 70, True, False)
school_data_complete['passingRead'] = np.where(school_data_complete.reading_score >= 70, True, False)


schSummarySetup = school_data_complete[['school_name', 'type','size','budget','math_score','reading_score',
                                        'passingMath','passingRead']]

schoolGroupBy = schSummarySetup.groupby(['school_name','type']) 
#include the reset_index to get the group by columns to display correct
schoolSummaryDF = pd.DataFrame(schoolGroupBy.mean()).reset_index()


#Add columns for Per Student Budget and Overall Passing Rate
schoolSummaryDF['Per_Student_Budget'] =schoolSummaryDF['budget']/schoolSummaryDF['size']
schoolSummaryDF['OverallPassingRate'] =(schoolSummaryDF['math_score']+schoolSummaryDF['reading_score'])/2

#Relabel columns
schoolSummaryDF = schoolSummaryDF.rename(columns={'school_name':'Name','type':'Type', 'size':'Student Count',
                                                  'budget':'Budget', 'math_score':'Avg Math Score',
                                                  'reading_score':'Avg Reading Score', 'passingMath':'% Passing Math',
                                                  'passingRead':'% Passing Reading', 'Per_Student_Budget':'Budget Per Student',
                                                 'OverallPassingRate':'Overall Pass Rate %'})
#Reorder columns
schoolSummaryDF = schoolSummaryDF[['Name','Type','Budget', 'Student Count','Budget Per Student','Avg Math Score',
                                   'Avg Reading Score','% Passing Math', '% Passing Reading','Overall Pass Rate %' ]]

#Created a data frame and reformat values
schoolSummaryDF_Fmt = schoolSummaryDF
schoolSummaryDF_Fmt['Budget'] = schoolSummaryDF_Fmt['Budget'].astype(float).apply('${:,.0f}'.format)
schoolSummaryDF_Fmt['Student Count'] = schoolSummaryDF_Fmt['Student Count'].astype(int)
schoolSummaryDF_Fmt['Budget Per Student']=round(schoolSummaryDF_Fmt['Budget Per Student'],1).apply('${:,.0f}'.format)
schoolSummaryDF_Fmt['Avg Math Score'] = round(schoolSummaryDF_Fmt['Avg Math Score'],1 )
schoolSummaryDF_Fmt['Avg Reading Score'] = round(schoolSummaryDF_Fmt['Avg Reading Score'],1 )
schoolSummaryDF_Fmt['% Passing Math']=(schoolSummaryDF_Fmt['% Passing Math']*100).apply('{:,.0f}%'.format)
schoolSummaryDF_Fmt['% Passing Reading']=(schoolSummaryDF_Fmt['% Passing Reading']*100).apply('{:,.0f}%'.format)
schoolSummaryDF_Fmt['Overall Pass Rate %']=round(schoolSummaryDF_Fmt['Overall Pass Rate %'],1).apply('{:,.0f}%'.format)


schoolSummaryDF_Fmt

,Name,Type,Budget,Student Count,Budget Per Student,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate %
0,Bailey High School,District,"$3,124,928",4976,$628,77.0,81.0,67%,82%,79%
1,Cabrera High School,Charter,"$1,081,356",1858,$582,83.1,84.0,94%,97%,84%
2,Figueroa High School,District,"$1,884,411",2949,$639,76.7,81.2,66%,81%,79%
3,Ford High School,District,"$1,763,916",2739,$644,77.1,80.7,68%,79%,79%
4,Griffin High School,Charter,"$917,500",1468,$625,83.4,83.8,93%,97%,84%
5,Hernandez High School,District,"$3,022,020",4635,$652,77.3,80.9,67%,81%,79%
6,Holden High School,Charter,"$248,087",427,$581,83.8,83.8,93%,96%,84%
7,Huang High School,District,"$1,910,635",2917,$655,76.6,81.2,66%,81%,79%
8,Johnson High School,District,"$3,094,650",4761,$650,77.1,81.0,66%,81%,79%
9,Pena High School,Charter,"$585,858",962,$609,83.8,84.0,95%,96%,84%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [45]:
schoolSummaryDF_Fmt.sort_values('Overall Pass Rate %', ascending = False).head()

,Name,Type,Budget,Student Count,Budget Per Student,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate %
1,Cabrera High School,Charter,"$1,081,356",1858,$582,83.1,84.0,94%,97%,84%
4,Griffin High School,Charter,"$917,500",1468,$625,83.4,83.8,93%,97%,84%
6,Holden High School,Charter,"$248,087",427,$581,83.8,83.8,93%,96%,84%
9,Pena High School,Charter,"$585,858",962,$609,83.8,84.0,95%,96%,84%
11,Shelton High School,Charter,"$1,056,600",1761,$600,83.4,83.7,94%,96%,84%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
#using second condtion of student count as tie breaker. Student count to give more weight to pass rate.
schoolSummaryDF_Fmt.sort_values(['Overall Pass Rate %','Student Count']).head()

,Name,Type,Budget,Student Count,Budget Per Student,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate %
3,Ford High School,District,"$1,763,916",2739,$644,77.1,80.7,68%,79%,79%
7,Huang High School,District,"$1,910,635",2917,$655,76.6,81.2,66%,81%,79%
2,Figueroa High School,District,"$1,884,411",2949,$639,76.7,81.2,66%,81%,79%
10,Rodriguez High School,District,"$2,547,363",3999,$637,76.8,80.7,66%,80%,79%
5,Hernandez High School,District,"$3,022,020",4635,$652,77.3,80.9,67%,81%,79%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [99]:
#wanted to explore using apply method
#already had passing math in school_data_complete df built using numpy where method, but wanted to try something else and
#only need student_data.
#will try lamba insted of creating named functionon reading version of this...

#define function for testing math score , reture true or false
def passing(n):
    return n >= 70

#create data frame for just the data needed and create column to denote if student is passing math or not
mathDf = student_data[['Student ID', 'school_name', 'grade', 'math_score']]
#used loc approach below per doc reco, to avoid the copy from slice warning
mathDf.loc[:,'PassingMath'] = mathDf['math_score'].apply(passing)

#pvttable the frame to shape and summarize data and set to a new data frame

mathDfPvt = mathDf.pivot_table(index = 'school_name', columns = 'grade', values = 'PassingMath').reset_index()
mathDfPvt.head()

#Orgainize and Format data for final presentation
mathBySchoolAndGrade = mathDfPvt[['school_name', '9th', '10th', '11th','12th']]
mathBySchoolAndGrade['9th']=round((mathBySchoolAndGrade['9th']*100),2).apply('{:,.0f}%'.format)
mathBySchoolAndGrade['10th']=round((mathBySchoolAndGrade['10th']*100),2).apply('{:,.0f}%'.format)
mathBySchoolAndGrade['11th']=round((mathBySchoolAndGrade['11th']*100),2).apply('{:,.0f}%'.format)
mathBySchoolAndGrade['12th']=round((mathBySchoolAndGrade['12th']*100),2).apply('{:,.0f}%'.format)

mathBySchoolAndGrade.head()



grade,school_name,9th,10th,11th,12th
0,Bailey High School,67%,66%,68%,64%
1,Cabrera High School,95%,94%,92%,95%
2,Figueroa High School,64%,67%,65%,69%
3,Ford High School,69%,69%,69%,65%
4,Griffin High School,92%,94%,94%,93%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [104]:
#try lamba insted of creating named functionon for passing reading column

#define function for testing math score , reture true or false
def passing(n):
    return n >= 70

#create data frame for just the data needed and create column to denote if student is passing math or not
readingDf = student_data[['Student ID', 'school_name', 'grade', 'reading_score']]
#used loc approach below per doc reco, to avoid the copy from slice warning
readingDf.loc[:,'PassingReading'] = readingDf['reading_score'].apply(passing)

#pvttable the frame to shape and summarize data and set to a new data frame

readingDfPvt = readingDf.pivot_table(index = 'school_name', columns = 'grade', values = 'PassingReading').reset_index()
readingDfPvt.head()

#Orgainize and Format data for final presentation
readingBySchoolAndGrade = readingDfPvt[['school_name', '9th', '10th', '11th','12th']]
readingBySchoolAndGrade['9th']=round((readingBySchoolAndGrade['9th']*100),2).apply('{:,.0f}%'.format)
readingBySchoolAndGrade['10th']=round((readingBySchoolAndGrade['10th']*100),2).apply('{:,.0f}%'.format)
readingBySchoolAndGrade['11th']=round((readingBySchoolAndGrade['11th']*100),2).apply('{:,.0f}%'.format)
readingBySchoolAndGrade['12th']=round((readingBySchoolAndGrade['12th']*100),2).apply('{:,.0f}%'.format)

readingBySchoolAndGrade.head()


grade,school_name,9th,10th,11th,12th
0,Bailey High School,82%,84%,81%,81%
1,Cabrera High School,97%,97%,97%,97%
2,Figueroa High School,82%,81%,78%,82%
3,Ford High School,80%,79%,80%,78%
4,Griffin High School,96%,97%,98%,98%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
